In [55]:
import os
from PIL import Image

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import copy

In [91]:
weights = torchvision.models.ResNet50_Weights.DEFAULT
model = torchvision.models.resnet50(weights=weights)

In [92]:
transform = weights.transforms()

dataset = ImageFolder(root='/Users/maillet/Desktop/Clean_Dirty/data/train', transform = transform)
print(len(dataset))

40


In [93]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
batch_size = 64

train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)

In [94]:
model.classifier = torch.nn.Sequential(torch.nn.Dropout(p=0.2, inplace=True),
                                       torch.nn.Linear(in_features=1024,
                                                       out_features=1,
                                                       bias=True
                                                      )
                                      )

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [95]:
def train_step(model, loader, criterion, optimizer, device):
    model.train()
    train_loss, correct_predictions, total_samples = 0, 0, 0
    for batch in loader:
        X,y = batch
        X, y = X.to(device), y.to(device)
        y_pred = model(X)
        error = criterion(y_pred, y)
        train_loss += error.item()

        optimizer.zero_grad()

        error.backward()

        optimizer.step()

        y_pred_class = torch.argmax(y_pred, dim=1)
        correct_predictions += (y_pred_class == y).sum().item()
        total_samples += len(y)
    
    avg_train_loss = train_loss / len(loader)
    train_acc = correct_predictions / total_samples
    return avg_train_loss, train_acc

In [96]:
def test_step(model, loader, criterion, device):
    model.eval()
    test_loss, correct_predictions, total_samples = 0, 0, 0
    with torch.no_grad():
        for batch in loader:
            X, y = batch
            X, y = X.to(device), y.to(device)
            y_pred = model(X)
            error = criterion(y_pred, y)
            test_loss += error.item()
            y_pred_class = torch.argmax(y_pred, dim=1)
            correct_predictions += (y_pred_class == y).sum().item()
            total_samples += len(y)  
    
    avg_test_loss = test_loss / len(loader)
    test_acc = correct_predictions / total_samples
    return avg_test_loss, test_acc

In [97]:
def fit(model, train_loader, optimizer, test_loader, criterion, device, epochs):
    results = {
        "train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
    }
    for epoch in range(epochs):
        avg_train_loss, train_acc = train_step(model, train_loader, criterion, optimizer, device)
        avg_test_loss, test_acc = test_step(model, test_loader, criterion, device)
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {avg_train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {avg_test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
        )
        results["train_loss"].append(avg_train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(avg_test_loss)
        results["test_acc"].append(test_acc)

In [98]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 15
fit(model, train_dataloader, optimizer, val_dataloader, criterion, device, epochs)

Epoch: 1 | train_loss: 7.3051 | train_acc: 0.0000 | test_loss: 4.7316 | test_acc: 0.1250
Epoch: 2 | train_loss: 4.7268 | train_acc: 0.3438 | test_loss: 2.2496 | test_acc: 0.5000
Epoch: 3 | train_loss: 2.3069 | train_acc: 0.5938 | test_loss: 1.5806 | test_acc: 0.5000
Epoch: 4 | train_loss: 0.8427 | train_acc: 0.9375 | test_loss: 1.2146 | test_acc: 0.5000
Epoch: 5 | train_loss: 0.2794 | train_acc: 0.9688 | test_loss: 1.1625 | test_acc: 0.6250
Epoch: 6 | train_loss: 0.0305 | train_acc: 1.0000 | test_loss: 1.0822 | test_acc: 0.7500
Epoch: 7 | train_loss: 0.0085 | train_acc: 1.0000 | test_loss: 0.9512 | test_acc: 0.7500
Epoch: 8 | train_loss: 0.0059 | train_acc: 1.0000 | test_loss: 0.8011 | test_acc: 0.8750
Epoch: 9 | train_loss: 0.0032 | train_acc: 1.0000 | test_loss: 0.7457 | test_acc: 0.8750
Epoch: 10 | train_loss: 0.0018 | train_acc: 1.0000 | test_loss: 0.7206 | test_acc: 0.8750
Epoch: 11 | train_loss: 0.0011 | train_acc: 1.0000 | test_loss: 0.6877 | test_acc: 0.8750
Epoch: 12 | train_l